In [2]:
import pandas as pd
import glob
import os
# read file from Bike share csv
path = r'D:\bootcamp\week_20_Final_Project\BIke sharing\Bike data'                    
all_files = glob.glob(os.path.join(path, "*.csv"))

# read each csv file and concatenate it into one dataframe
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [3]:
concatenated_df.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,4581278,1547.0,7021,01/01/2019 00:08,Bay St / Albert St,7233,01/01/2019 00:33,King / Cowan Ave - SMART,1296,Annual Member
1,4581279,1112.0,7160,01/01/2019 00:10,King St W / Tecumseth St,7051,01/01/2019 00:29,Wellesley St E / Yonge St (Green P),2947,Annual Member
2,4581280,589.0,7055,01/01/2019 00:15,Jarvis St / Carlton St,7013,01/01/2019 00:25,Scott St / The Esplanade,2293,Annual Member
3,4581281,259.0,7012,01/01/2019 00:16,Elizabeth St / Edward St (Bus Terminal),7235,01/01/2019 00:20,Bay St / College St (West Side) - SMART,283,Annual Member
4,4581282,281.0,7041,01/01/2019 00:19,Edward St / Yonge St,7257,01/01/2019 00:24,Dundas St W / St. Patrick St,1799,Annual Member


In [4]:
#delete na row

concatenated_df.dropna(subset = ['User Type'], inplace=True)#249 rows

In [5]:
#check if there is still na data
concatenated_df[concatenated_df['User Type'].isna()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type


In [6]:
# change datetype from datetime to date
concatenated_df['Date']=pd.to_datetime(concatenated_df['Start Time']).dt.date

In [7]:
Trips=concatenated_df.groupby(['Date'])['Trip Id'].count().reset_index(name='counts_trips')

In [8]:
Bikes=concatenated_df.groupby(['Date'])['Bike Id'].nunique().reset_index(name='counts_Bike')

In [9]:
Ann_df0 = concatenated_df[concatenated_df['User Type'] == 'Annual Member']
Ann_df=Ann_df0.groupby(['Date'])['Trip Id'].count().reset_index(name='counts_member_annual')

In [10]:
cas_df0 = concatenated_df[concatenated_df['User Type'] == 'Casual Member']
cas_df=cas_df0.groupby(['Date'])['Trip Id'].count().reset_index(name='counts_member_casual')

In [11]:
df_tb = pd.merge(Trips,Bikes,on=['Date'])
df_tba= pd.merge(df_tb,Ann_df,on=['Date'])
df_final= pd.merge(df_tba,cas_df,on=['Date'])

In [12]:
df_final

,Date,counts_trips,counts_Bike,counts_member_annual,counts_member_casual
0,2019-01-01,1064,674,897,167
1,2019-01-02,2290,1135,2207,83
2,2019-01-03,2279,955,2228,51
3,2019-01-04,3290,1381,3130,160
4,2019-01-05,2160,1053,1937,223
...,...,...,...,...,...
756,2021-12-27,1265,756,873,392
757,2021-12-28,3054,1466,2025,1029
758,2021-12-29,3521,1668,2440,1081
759,2021-12-30,4065,1785,2836,1229


In [13]:
#sum trip duration group by date
td_df=concatenated_df.groupby(['Date'])['Trip  Duration'].sum().reset_index(name='trip_duration')
#merge to final dataframe
df_final2= pd.merge(df_final,td_df,on=['Date'])

In [14]:
#change column order
df_final2=df_final2[['Date','counts_trips','counts_Bike','trip_duration','counts_member_annual','counts_member_casual']]

df_final2

,Date,counts_trips,counts_Bike,trip_duration,counts_member_annual,counts_member_casual
0,2019-01-01,1064,674,1121463.0,897,167
1,2019-01-02,2290,1135,2838818.0,2207,83
2,2019-01-03,2279,955,1650338.0,2228,51
3,2019-01-04,3290,1381,2269606.0,3130,160
4,2019-01-05,2160,1053,1468872.0,1937,223
...,...,...,...,...,...,...
756,2021-12-27,1265,756,955753.0,873,392
757,2021-12-28,3054,1466,2356616.0,2025,1029
758,2021-12-29,3521,1668,2896184.0,2440,1081
759,2021-12-30,4065,1785,2938443.0,2836,1229


In [15]:
#save to csv
df_final2.to_csv('result/Bike_data.csv',index=False)

In [16]:
#merge data weather and bike
weather_data_df=pd.read_csv('weather/weather_data_clean.csv')
weather_data_df

,W_date,W_Temp_Max,W_Temp_Avg,W_Temp_Min,W_Max_wind,W_Avg_wind,W_Min_wind,W_Wind_Gust,W_Air_Pressure,W_Snow_Depth,W_Max_humid,W_Avg_humid,W_Min_humid,W_Max_Dp,W_Avg_dp,W_Min_Dp
0,2019-01-01,5.2,-1.4,-8.0,37.0,21.4,3.2,58.0,1015.1,NaN,100.0,85.5,64.0,6.1,-0.6,-11.1
1,2019-01-02,-0.5,-4.9,-9.3,20.9,11.9,8.0,NaN,1023.0,0.0,100.0,84.0,74.0,0.0,-5.1,-11.1
2,2019-01-03,0.8,-1.0,-2.8,37.0,18.2,0.0,46.0,1013.3,30.0,100.0,86.0,69.0,0.0,-1.9,-5.0
3,2019-01-04,8.0,3.6,-0.8,29.0,18.8,8.0,54.0,1007.6,30.0,87.0,80.7,70.0,2.2,0.1,-1.1
4,2019-01-05,4.0,1.2,-1.7,32.2,16.3,3.2,33.0,1006.1,0.0,93.0,83.7,75.0,1.1,-0.1,-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27,0.1,-3.1,-6.2,59.5,38.9,9.7,48.0,1014.1,40.0,100.0,90.9,68.0,1.1,-2.3,-10.0
1092,2021-12-28,2.3,0.9,-0.6,40.2,17.2,3.2,57.0,1009.8,20.0,100.0,80.5,56.0,2.2,-1.4,-5.0
1093,2021-12-29,2.2,1.0,-0.3,32.2,20.8,4.8,NaN,1011.7,10.0,100.0,91.1,81.0,2.8,0.9,-1.1
1094,2021-12-30,2.2,1.1,0.0,24.1,12.6,4.8,NaN,1013.8,NaN,87.0,80.5,70.0,1.1,-0.4,-2.2


In [23]:
df_final2.Date = pd.to_datetime(df_final2.Date)
weather_data_df.W_date = pd.to_datetime(weather_data_df.W_date)

Merge_data_df=pd.merge(df_final2, weather_data_df, left_on='Date', right_on='W_date')

In [24]:
Merge_data_df = Merge_data_df.drop(['W_date'], 1)

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [25]:
#save to csv
Merge_data_df.to_csv('result/Bike_weather_merge.csv',index=False)

In [26]:
weather_data_df.dtypes

W_date            datetime64[ns]
W_Temp_Max               float64
W_Temp_Avg               float64
W_Temp_Min               float64
W_Max_wind               float64
W_Avg_wind               float64
W_Min_wind               float64
W_Wind_Gust              float64
W_Air_Pressure           float64
W_Snow_Depth             float64
W_Max_humid              float64
W_Avg_humid              float64
W_Min_humid              float64
W_Max_Dp                 float64
W_Avg_dp                 float64
W_Min_Dp                 float64
dtype: object

In [27]:
df_final2.dtypes

Date                    datetime64[ns]
counts_trips                     int64
counts_Bike                      int64
trip_duration                  float64
counts_member_annual             int64
counts_member_casual             int64
dtype: object